In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/obesity-levels/ObesityDataSet_raw_and_data_sinthetic.csv")

Questions:Possible Answers

¿What is your gender?
* Female
* Male

¿what is your age?	Numeric value

¿what is your height?	Numeric value in meters

¿what is your weight?	Numeric value in kilograms

¿Has a family member suffered or suffers from overweight?
* Yes
* No

¿Do you eat high caloric food frequently?	
* Yes
* No

¿Do you usually eat vegetables in your meals?	
* Never
* Sometimes
* Always

¿How many main meals do you have daily?	
* Between 1 y 2
* Three
* More than three

¿Do you eat any food between meals?	

* No
* Sometimes
* Frequently
* Always

¿Do you smoke?	

* Yes
* No

¿How much water do you drink daily?	

* Less than a liter
* Between 1 and 2 L
* More than 2 L

¿Do you monitor the calories you eat daily?	

* Yes
* No

¿How often do you have physical activity?	

* I do not have
* 1 or 2 days
* 2 or 4 days
* 4 or 5 days

¿How much time do you use technological devices such as cell phone, videogames, television, computer and others?	

* 0–2 hours
* 3–5 hours
* More than 5 hours

¿how often do you drink alcohol?	

* I do not drink
* Sometimes
* Frequently
* Always

¿Which transportation do you usually use?	

* Automobile
* Motorbike
* Bike
* Public Transportation
* Walking

In [ ]:
data.head()

In [ ]:
# get the years
gender = data['Gender'].values
# get the values 
obs = data['NObeyesdad'].values

# create
data["Gender"].value_counts().plot(kind='bar')
#plt.show()

In [ ]:
pd.DataFrame(data[["MTRANS","NObeyesdad"]].groupby("MTRANS")["NObeyesdad"].value_counts().rename_axis(["MTRANS",'NObeyesdad_'])).reset_index().pivot(index='NObeyesdad_', columns='MTRANS', values='NObeyesdad').plot(kind="bar",stacked = True)

In [ ]:
pd.DataFrame(data[["family_history_with_overweight","NObeyesdad"]].groupby("family_history_with_overweight")["NObeyesdad"].value_counts().rename_axis(["family_history_with_overweight",'NObeyesdad_'])).reset_index().pivot(index='NObeyesdad_', columns='family_history_with_overweight', values='NObeyesdad').plot(kind="bar",stacked = True)

In [ ]:
data.corr()

In [ ]:
data['family_history_with_overweight'].unique()

In [ ]:
def convertyesandnow(str):
    if str == 'yes':
        return 1
    else:
        return 0

    
data['family_history_with_overweight_num'] = data['family_history_with_overweight'].apply(convertyesandnow)
data['FAVC_num'] = data['FAVC'].apply(convertyesandnow)
data['SMOKE_num'] = data['SMOKE'].apply(convertyesandnow)
data['SCC_num'] = data['SCC'].apply(convertyesandnow)

In [ ]:
data['CAEC'].unique()

In [ ]:
def convertyesandnow(str):
    if str == 'Always':
        return 10
    elif str == 'Frequently':
        return 6
    elif str == 'Sometimes':
        return 3
    else:
        return 0

    
data['CAEC_num'] = data['CAEC'].apply(convertyesandnow)
data['CALC_num'] = data['CALC'].apply(convertyesandnow)

In [ ]:
def convertgender(str):
    if str == 'Male':
        return 1
    else:
        return 0

    
data['Gender_num'] = data['Gender'].apply(convertgender)

In [ ]:
data['MTRANS'].unique()

In [ ]:
def convertMTRANS(str):
    if str == 'Automobile':
        return 4
    elif str == 'Motorbike':
        return 3
    elif str == 'Bike':
        return 2
    elif str == 'Public_Transportation':
        return 1
    else:
        return 0

    
data['MTRANS_num'] = data['MTRANS'].apply(convertMTRANS)

In [ ]:

data['NObeyesdad'].unique()

In [ ]:
data

In [ ]:
data.corr()

We see the family history of overweight is highly correlated with weight of the person

In [ ]:
data.columns
data[['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE','family_history_with_overweight_num','FAVC_num','SMOKE_num','SCC_num','CAEC_num','CALC_num','Gender_num','MTRANS_num']]

In [ ]:
#X = StandardScaler().fit_transform(data[['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE','family_history_with_overweight_num','FAVC_num','SMOKE_num','SCC_num','CAEC_num','CALC_num','Gender_num','MTRANS_num']])
#X
X = data[['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE','family_history_with_overweight_num','FAVC_num','SMOKE_num','SCC_num','CAEC_num','CALC_num','Gender_num','MTRANS_num']]

In [ ]:
y = data['NObeyesdad']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

In [ ]:
obesity_classifier = DecisionTreeClassifier(max_leaf_nodes=13, random_state=0)
obesity_classifier.fit(X_train, y_train)

In [ ]:
predictions = obesity_classifier.predict(X_test)

In [ ]:
accuracy_score(y_true = y_test, y_pred = predictions)

In [ ]:
y.unique()

In [ ]:
fig = plt.figure(figsize=(25,20))
tree.plot_tree(obesity_classifier,feature_names=X.columns,  
                   class_names=y.unique(),filled = True)